In [4]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium.common.exceptions import TimeoutException

import warnings
import pandas as pd
import numpy as np
import os

warnings.filterwarnings('ignore')

In [97]:
def click_next_button(driver):
    try:
        wait = WebDriverWait(driver, 10)
        next_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[@aria-label='View next page']")))
        next_button.click()
        print("✅ Clicked 'Next' button.")
        return True
    except TimeoutException:
        print("❌ 'Next' button not found or not clickable.")
        return False

import time

def get_scroll_container(driver):
    wait = WebDriverWait(driver, 15)
    try:
        return wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.jobs-search-results-list")))
    except:
        return wait.until(EC.presence_of_element_located((By.XPATH, "//div[@aria-label='Job list']")))

def scroll_left_panel(driver, max_scrolls=30):
    scroll_container = get_scroll_container(driver)
    driver.execute_script("arguments[0].style.border='2px solid red'", scroll_container)  # debug

    last_height = driver.execute_script("return arguments[0].scrollHeight", scroll_container)

    for i in range(max_scrolls):
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scroll_container)
        time.sleep(2)
        new_height = driver.execute_script("return arguments[0].scrollHeight", scroll_container)
        if new_height == last_height:
            print("✅ Reached bottom.")
            break
        last_height = new_height


In [ ]:
# 🔐 Your login credentials (replace with your actual username/password)
EMAIL = "
PASSWORD = ""


# Target URL
login_url = "https://www.linkedin.com/login"  # Login page


# Page counter
page_number = 1

# while page_number <= 16:
# while True:
base_url = "https://www.linkedin.com/login"
# Setup Selenium WebDriver
service = Service(ChromeDriverManager().install())
options = webdriver.ChromeOptions()
options.add_experimental_option("detach", True)  # Keeps the browser open
driver = webdriver.Chrome(service=service, options=options)
driver.get(base_url)

# try:
# WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.NAME, "Email")))
# button = driver.find_element(By.CLASS_NAME, "btn a_signin")
login_xpath = "//*[@id='username']"
password_xpath = "//*[@id='organic-div']/form/div[2]/label"
sign_in_xpath = "//*[@id='organic-div']/form/div[4]/button"
# button = driver.find_element(By.XPATH, login_xpath)
# button.click()
time.sleep(3)
driver.find_element(By.XPATH, login_xpath).send_keys(EMAIL)
driver.find_element(By.ID, "password").send_keys(PASSWORD)
driver.find_element(By.XPATH, sign_in_xpath).click()
print("✅ Logged in successfully!")

# Wait for login to complete
time.sleep(15)

# for keyword in ['data scientist','software engineer','data analyst','product scientist','ml engineer','ai engineer','gen ai engineer']:
for keyword in ['ml engineer','ai engineer','gen ai engineer']:
    home_xpath = '//*[@id="global-nav"]/div/nav/ul/li[1]/a/div/div/li-icon'
    driver.find_element(By.XPATH, home_xpath).click()
    
    print(f'Searching for {keyword}!')
    search_xpath = "//input[@aria-label='Search']"
    search = driver.find_element(By.XPATH,search_xpath)
    search.click()
    search.send_keys(keyword)
    search.send_keys(Keys.ENTER)
    
    time.sleep(5)
    ## Click on jobs button to get all the jobs list
    driver.find_element(By.XPATH, '//*[@id="search-reusables__filters-bar"]/ul/li[1]/button').click()
    # driver.find_element(By.ID, 'Jobs').click()
    # '//*[@id="search-reusables__filters-bar"]/ul/li[3]/button'
    # jobs_button = WebDriverWait(driver, 10).until(
    #         EC.element_to_be_clickable((By.XPATH, "//button[normalize-space(text())='Jobs']"))
    #     )
    # jobs_button.click()
    
    job_data = pd.DataFrame(columns=['job_name','company','location','joburl'])
    
    for i in range(5):
        time.sleep(5)
        
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        # Wait for at least one job card to load
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "div.job-card-container"))
        )
        
        job_cards = driver.find_elements(By.CSS_SELECTOR, "div.job-card-container")
        
        print(f"Found {len(job_cards)} jobs.")
        
        time.sleep(3)
        for i, job in enumerate(job_cards, 1):
            try:
                # Title
                title_element = job.find_element(By.CSS_SELECTOR, "a.job-card-container__link")
                title = title_element.text.strip()
                title = [x for x in title.split('\n')][0].strip()
        
                # Job URL
                job_url = title_element.get_attribute("href")
        
                # Company Name
                company = job.find_element(By.CSS_SELECTOR, "div.artdeco-entity-lockup__subtitle span").text.strip()
        
                # Location
                location = job.find_element(By.CSS_SELECTOR, "ul.job-card-container__metadata-wrapper li").text.strip()
        
                job_data.loc[len(job_data)] = [title, company, location, job_url]
        
            except Exception as e:
                print(f"{i}. ❌ Error extracting job: {e}")
        job_data.to_parquet(f"./data/jobs_{keyword.replace(' ','')}.parquet")
        success = click_next_button(driver)
    print(f"{keyword} done!")
    # except:
    #     print("Could not find")
    
    # except Exception as e:
    #     print(f"❌ Login failed: {e}")
    #     driver.quit()
    #     exit()

✅ Logged in successfully!
Searching for ml engineer!
Found 25 jobs.
✅ Clicked 'Next' button.
Found 25 jobs.
✅ Clicked 'Next' button.
Found 25 jobs.
✅ Clicked 'Next' button.
Found 25 jobs.
✅ Clicked 'Next' button.
Found 25 jobs.
✅ Clicked 'Next' button.
ml engineer done!
Searching for ai engineer!
Found 25 jobs.
✅ Clicked 'Next' button.
Found 25 jobs.
✅ Clicked 'Next' button.
Found 25 jobs.
✅ Clicked 'Next' button.
Found 25 jobs.
✅ Clicked 'Next' button.
Found 25 jobs.
✅ Clicked 'Next' button.
ai engineer done!
Searching for gen ai engineer!
Found 25 jobs.
✅ Clicked 'Next' button.
Found 25 jobs.
✅ Clicked 'Next' button.
Found 25 jobs.
✅ Clicked 'Next' button.
Found 25 jobs.
✅ Clicked 'Next' button.
Found 25 jobs.
✅ Clicked 'Next' button.
gen ai engineer done!


In [115]:
jobs = pd.DataFrame(columns=['job_name','company','location','joburl'])

files = os.listdir("./data")
for f in files:
    data = pd.read_parquet(f"./data/{f}")
    jobs = pd.concat([jobs, data],axis=0)

jobs.shape

(875, 4)

In [117]:
jobs.head(3)

,job_name,company,location,joburl
0,Machine Learning Engineer II,Atlassian,"Bengaluru, Karnataka, India (Remote)",https://www.linkedin.com/jobs/view/4219958889/...
1,Senior Machine Learning Engineer - II,Adobe,"Noida, Uttar Pradesh, India",https://www.linkedin.com/jobs/view/4223614905/...
2,Senior Machine Learning Engineer,Target,"Bengaluru, Karnataka, India (Hybrid)",https://www.linkedin.com/jobs/view/4215587397/...


In [118]:
jobs.to_parquet("./all_jobs_db.parquet")

In [5]:
jobs = pd.read_parquet("./all_jobs_db.parquet")

In [121]:
from strsimpy.normalized_levenshtein import NormalizedLevenshtein

In [126]:
job_list = jobs[['job_name']].drop_duplicates()

In [134]:
def findSimilarity(w1, w2):
    normalized_levenshtein = NormalizedLevenshtein()

    return normalized_levenshtein.similarity(w1, w2)

# @app.get("/match-medicine/")
def match_jobs(query: str):
    """
    Finds the top 3 similar medicine names based on the input query.
    """

    all_simi = pd.DataFrame(columns=['input','match','score'])
    for idx, row in job_list.iterrows():
        w1 = query
        for col in ['job_name']:
            w2 = row[col]
            simi = findSimilarity(w1, w2.lower())
            all_simi.loc[len(all_simi)] = [w1, w2, simi]
    matches = all_simi.sort_values('score',ascending=False).head(10)[['match','score']].values
    return [
        {"job_name": match[0], "similarity_score": np.round(match[1],3)}
        for match in matches
    ]

In [135]:
match_jobs('data scientist')

[{'job_name': 'Data Scientist', 'similarity_score': 1.0},
 {'job_name': 'Data Scientist 2', 'similarity_score': 0.875},
 {'job_name': 'Sr Data Scientist', 'similarity_score': 0.824},
 {'job_name': 'Data Scientist II', 'similarity_score': 0.824},
 {'job_name': 'AI Scientist', 'similarity_score': 0.786},
 {'job_name': 'Data Scientist III', 'similarity_score': 0.778},
 {'job_name': 'Data Scientist - 2', 'similarity_score': 0.778},
 {'job_name': 'Sr. Data Scientist', 'similarity_score': 0.778},
 {'job_name': 'Data Scientist , VP', 'similarity_score': 0.737},
 {'job_name': 'Lead Data Scientist', 'similarity_score': 0.737}]

In [136]:
jobs

,job_name,company,location,joburl
0,Machine Learning Engineer II,Atlassian,"Bengaluru, Karnataka, India (Remote)",https://www.linkedin.com/jobs/view/4219958889/...
1,Senior Machine Learning Engineer - II,Adobe,"Noida, Uttar Pradesh, India",https://www.linkedin.com/jobs/view/4223614905/...
2,Senior Machine Learning Engineer,Target,"Bengaluru, Karnataka, India (Hybrid)",https://www.linkedin.com/jobs/view/4215587397/...
3,Senior ML Engineer,Razorpay,"Bengaluru, Karnataka, India (On-site)",https://www.linkedin.com/jobs/view/4222071037/...
4,Machine Learning Engineer (T25),eBay,"Bengaluru, Karnataka, India (Hybrid)",https://www.linkedin.com/jobs/view/4218288595/...
...,...,...,...,...
120,"AI Engineer, AVP",Deutsche Bank,"Pune, Maharashtra, India (Hybrid)",https://www.linkedin.com/jobs/view/4230479554/...
121,Senior AI Engineer,Lowe's India,"Bengaluru, Karnataka, India (Hybrid)",https://www.linkedin.com/jobs/view/4226978027/...
122,"Senior Data Engineer, Global Converse, ITC",Nike,"Karnataka, India (On-site)",https://www.linkedin.com/jobs/view/4227092908/...
123,"Deep Learning Engineer, Datacenters",NVIDIA,"Bengaluru, Karnataka, India (Hybrid)",https://www.linkedin.com/jobs/view/4206277697/...


In [137]:
from gensim.models import word2vec

In [147]:
all_texts_train = jobs['job_name'].values

training_words = list(map(lambda x: x.lower().split(), all_texts_train))
train_words = []
for x in training_words:
    train_words = train_words + x
print(len(train_words), train_words[:5])

3780 ['machine', 'learning', 'engineer', 'ii', 'senior']


In [158]:
%%time

vector_size = 100

word2vec_model = word2vec.Word2Vec(training_words,
                 vector_size=256,
                 workers=8,
                 min_count=1)

print("Vocabulary Length:", len(word2vec_model.wv.key_to_index))

Vocabulary Length: 319
CPU times: user 32.8 ms, sys: 5.95 ms, total: 38.8 ms
Wall time: 34.3 ms


In [159]:
word2vec_model.save("./word2vec_model/word2vec_trained_model.bin")

In [160]:
model = word2vec.Word2Vec.load("./word2vec_model/word2vec_trained_model.bin")

In [165]:
model.wv['data'].shape

(256,)

In [169]:
embedding_df = pd.DataFrame()
for idx, row in jobs.iterrows():
    tensor = np.zeros(256)
    for w in row['job_name'].split(" "):
        tensor = tensor + model.wv[w.lower()]
    embedding_df[row['job_name']] = tensor

In [170]:
embedding_df

,Machine Learning Engineer II,Senior Machine Learning Engineer - II,Senior Machine Learning Engineer,Senior ML Engineer,Machine Learning Engineer (T25),Machine Learning Engineer,Sr ML Engineer,Data Insights and Analytics Engineer,Machine Learning Engineer III (Recommender Systems),"Software Engineer, Machine Learning",...,Applied AI ML Associate Senior,"Software Development , Amazon",GenAI Team Lead,Senior Engineer - Data Science Analytics,Computer Scientist - II,"Data Engineer, Global Converse, ITC",Assistant VP - Quantitative Analytics,"Senior Data Engineer, Global Converse, ITC","Deep Learning Engineer, Datacenters",Software Engineer III
0,-0.007518,-0.008453,-0.012391,-0.003423,-0.011484,-0.010567,-0.001983,-0.006146,-0.012143,-0.009195,...,0.003458,-0.005272,-0.001127,-0.007574,0.002167,-0.002701,-0.000735,-0.004525,-0.006718,-0.001863
1,-0.038473,-0.081675,-0.053454,-0.049586,-0.041347,-0.038439,-0.040545,-0.065570,-0.045766,-0.041967,...,-0.069442,-0.023551,-0.025681,-0.100670,-0.049411,-0.040707,-0.047863,-0.055722,-0.024938,-0.030150
2,-0.013904,-0.023107,-0.011432,-0.008065,-0.006990,-0.010314,-0.005411,-0.007372,-0.008667,-0.005604,...,-0.007479,-0.008145,-0.004437,-0.016312,-0.017033,-0.003027,-0.008926,-0.004145,-0.010614,-0.003245
3,0.029742,0.056263,0.033188,0.027087,0.027645,0.025844,0.024942,0.041619,0.030626,0.029034,...,0.043195,0.012034,0.015455,0.064176,0.030976,0.027892,0.036169,0.035236,0.017398,0.020581
4,0.046540,0.090053,0.057476,0.051425,0.042228,0.042933,0.042028,0.064822,0.057034,0.044980,...,0.073123,0.024306,0.018302,0.106608,0.043727,0.040845,0.050434,0.055388,0.034314,0.033219
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,0.013770,0.026009,0.015577,0.014331,0.016312,0.014039,0.015363,0.028710,0.014032,0.016002,...,0.023932,0.014612,0.014111,0.040502,0.021352,0.009505,0.022423,0.011043,0.010534,0.011080
252,-0.018905,-0.033776,-0.025202,-0.020451,-0.019867,-0.018284,-0.014567,-0.036981,-0.019468,-0.014524,...,-0.024009,-0.012467,-0.012012,-0.049931,-0.012080,-0.022141,-0.006609,-0.029059,-0.011716,-0.010693
253,-0.016372,-0.032015,-0.023900,-0.017608,-0.014357,-0.017866,-0.013250,-0.023014,-0.024319,-0.014513,...,-0.028367,-0.013142,-0.004399,-0.043377,-0.011284,-0.021353,-0.018115,-0.027387,-0.009753,-0.018824
254,-0.045810,-0.071805,-0.050146,-0.040621,-0.045687,-0.042118,-0.039503,-0.064645,-0.050578,-0.037732,...,-0.062100,-0.022018,-0.010022,-0.091055,-0.038526,-0.039959,-0.036554,-0.047987,-0.024843,-0.025851


In [171]:
embedding_df.to_parquet("./data/embeddings_word2vec.parquet")

In [172]:
from sklearn.metrics.pairwise import cosine_similarity

In [182]:
w1 = np.array(embedding_df[embedding_df.columns[0]])#.reshape(-1,1)
w2 = np.array(embedding_df[embedding_df.columns[1]])#.reshape(-1,1)
# cosine_similarity(w1, w2)

In [181]:
np.dot(w1, w2)/(np.linalg.norm(w1)*np.linalg.norm(w2))

0.9941755665264399

In [185]:
'data' in model.wv.key_to_index.keys()

True

In [2]:
from sentence_transformers import SentenceTransformer

# Load a pre-trained sentence transformer model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Your sentence
sentence = "Data science is an interdisciplinary field."

# Generate the embedding
embedding = model.encode(sentence)

# Display the vector (embedding)
print("Embedding shape:", embedding.shape)
print("Embedding vector:", embedding)


/Users/vishaldawar/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/vishaldawar/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Embedding shape: (384,)
Embedding vector: [ 7.83491414e-03 -2.18882151e-02 -1.35582248e-02 -1.22825440e-03
 -6.10824190e-02 -1.23834215e-01 -6.27454137e-03 -1.76162757e-02
 -2.49444805e-02  8.41226652e-02 -6.15137257e-02  1.59966256e-02
  4.28882912e-02 -8.83933622e-03 -2.51346966e-03  5.58755808e-02
 -5.67491502e-02 -5.66407703e-02 -4.18168865e-02 -2.56953370e-02
 -5.23954816e-02  2.61697099e-02 -4.00324352e-02 -3.10432427e-02
  4.26853150e-02  4.33528982e-02  6.70114458e-02  1.08307853e-04
 -2.44315714e-02 -3.65521386e-02 -5.77164367e-02  1.35182500e-01
  7.08198035e-03  6.66466951e-02 -4.05074060e-02  2.35587154e-02
  9.47028548e-02  3.22811259e-03  6.04112707e-02  6.59507960e-02
 -3.31466720e-02  8.90632533e-03  2.27789097e-02 -4.26385785e-03
 -3.01930383e-02 -1.42646795e-02 -3.03275250e-02 -1.33586526e-01
  3.57522592e-02  1.96374441e-03 -9.54786092e-02  1.17602069e-02
 -1.95497777e-02  3.42641845e-02  1.44540407e-02  4.01995657e-03
  1.01461701e-01  4.61738706e-02 -4.13082428e-02

In [6]:
embedding_llm = pd.DataFrame()
for idx, row in jobs.iterrows():
    w = row['job_name']
    tensor = model.encode(w)
    embedding_llm[row['job_name']] = tensor

In [7]:
embedding_llm

,Machine Learning Engineer II,Senior Machine Learning Engineer - II,Senior Machine Learning Engineer,Senior ML Engineer,Machine Learning Engineer (T25),Machine Learning Engineer,Sr ML Engineer,Data Insights and Analytics Engineer,Machine Learning Engineer III (Recommender Systems),"Software Engineer, Machine Learning",...,Applied AI ML Associate Senior,"Software Development , Amazon",GenAI Team Lead,Senior Engineer - Data Science Analytics,Computer Scientist - II,"Data Engineer, Global Converse, ITC",Assistant VP - Quantitative Analytics,"Senior Data Engineer, Global Converse, ITC","Deep Learning Engineer, Datacenters",Software Engineer III
0,-0.088197,-0.064287,-0.049640,-0.057628,-0.068801,-0.070650,-0.099967,-0.030083,-0.082825,-0.055439,...,-0.041215,0.017605,-0.061587,-0.002357,-0.071027,-0.095659,-0.063716,-0.080730,-0.091979,-0.083171
1,-0.047135,0.007855,0.002369,0.012200,-0.049168,-0.039121,-0.011463,0.014292,-0.110414,-0.017124,...,-0.069851,-0.056548,0.065469,0.074055,0.038543,-0.036102,-0.007824,0.006768,-0.083194,-0.021346
2,0.059874,0.074023,0.065368,0.039561,0.055907,0.052644,-0.018302,-0.034524,0.014447,0.049458,...,0.020530,-0.034373,-0.057116,-0.004744,-0.022779,0.022399,-0.040714,0.035510,0.111101,-0.003499
3,0.031611,0.005362,0.011442,-0.013918,-0.011388,0.046966,0.031150,0.095157,0.045056,0.027872,...,-0.030844,0.016289,-0.000051,0.050936,0.025661,0.011310,0.024810,0.013608,0.051487,-0.016892
4,0.015481,0.016792,0.005301,-0.060777,0.030963,-0.012865,-0.044411,0.011827,0.050693,0.007479,...,-0.011986,0.051569,0.017371,0.031038,-0.045608,-0.002756,0.014115,-0.023861,-0.052331,-0.015113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,0.022869,-0.008306,-0.000789,-0.024398,0.027112,0.045087,0.008011,0.014217,0.000501,0.043406,...,-0.017560,0.085668,0.011843,-0.040533,0.076448,0.088330,0.071495,0.042442,0.095185,0.019980
380,0.067987,0.143195,0.131959,0.116151,0.018488,0.041742,0.091948,0.060804,0.055617,-0.001827,...,0.102868,-0.001006,-0.016494,0.109312,0.126236,-0.018838,0.024838,0.057909,0.004709,0.071344
381,-0.006248,-0.040497,-0.063841,-0.066906,-0.023150,-0.020195,-0.048827,-0.043914,-0.011166,-0.028133,...,-0.035794,-0.066455,0.002714,-0.064523,-0.017578,-0.044583,-0.063316,-0.070145,-0.058233,-0.066578
382,-0.042903,-0.051279,-0.005254,0.059114,-0.015982,0.007271,0.024666,0.014362,-0.049517,0.019911,...,-0.025284,0.035627,0.023521,0.046362,-0.016358,-0.059686,-0.032788,-0.064356,-0.079170,0.032436


In [8]:
embedding_llm.to_parquet("./data/embeddings_llm.parquet")

In [9]:
w1 = np.array(embedding_llm[embedding_llm.columns[0]])#.reshape(-1,1)
w2 = np.array(embedding_llm[embedding_llm.columns[1]])#.reshape(-1,1)
np.dot(w1, w2)/(np.linalg.norm(w1)*np.linalg.norm(w2))

0.851888